# Library

In [1]:
import pandas as pd
from dataset import Dataset
from dataread import Dataread
import seaborn as sns
import matplotlib.pyplot as plt
import os
import fsspec
import ast

sns.set_style("whitegrid")
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False


ds = Dataset()
dr = Dataread()
JSON_KEY_PATH = "./sprintda05-soomin.json"
os.environ["GOOGlE_APPLICATION_CREDENTIALS"] = JSON_KEY_PATH
fs = fsspec.filesystem("gcs")

# 가입 후 n일 이내 결제 유무

In [5]:
ds.votes.file_list()

['accounts_attendance',
 'accounts_blockrecord',
 'accounts_failpaymenthistory',
 'accounts_friendrequest',
 'accounts_group',
 'accounts_nearbyschool',
 'accounts_paymenthistory',
 'accounts_pointhistory',
 'accounts_school',
 'accounts_timelinereport',
 'accounts_user',
 'accounts_user_contacts',
 'accounts_userquestionrecord',
 'accounts_userwithdraw',
 'event_receipts',
 'events',
 'polls_question',
 'polls_questionpiece',
 'polls_questionreport',
 'polls_questionset',
 'polls_usercandidate']

In [33]:
df = (ds.votes.accounts_userquestionrecord()
    .query("created_at < '2023-05-26' and created_at >= '2023-04-28'")
    .assign(day = lambda df: pd.to_datetime(df['created_at']).dt.floor('D'))
    )          
df                                

,id,status,created_at,chosen_user_id,question_id,user_id,question_piece_id,has_read,answer_status,answer_updated_at,report_count,opened_times,day
0,771777,C,2023-04-28 12:27:49,849469,252,849436,998458,0,N,2023-04-28 12:27:49,0,0,2023-04-28
1,771800,C,2023-04-28 12:28:02,849446,244,849436,998459,0,N,2023-04-28 12:28:02,0,0,2023-04-28
2,771812,C,2023-04-28 12:28:09,849454,183,849436,998460,1,N,2023-04-28 12:28:09,0,0,2023-04-28
3,771828,C,2023-04-28 12:28:16,847375,101,849436,998461,0,N,2023-04-28 12:28:16,0,0,2023-04-28
4,771851,C,2023-04-28 12:28:26,849477,209,849436,998462,1,N,2023-04-28 12:28:26,0,0,2023-04-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015466,112613845,C,2023-05-25 23:59:21,1449191,530,1471840,142402269,0,N,2023-05-25 23:59:21,0,0,2023-05-25
1015467,112613858,C,2023-05-25 23:59:21,1066708,241,1371253,134697457,1,N,2023-05-25 23:59:21,0,0,2023-05-25
1015468,112614655,C,2023-05-25 23:59:46,1014562,1609,1177576,143571050,1,N,2023-05-25 23:59:46,0,0,2023-05-25
1015469,112614805,C,2023-05-25 23:59:51,1063812,605,1177576,143571051,0,N,2023-05-25 23:59:51,0,0,2023-05-25


In [ ]:
# 최초 날짜 로직
df_sorted = df.sort_values(by=['user_id','day'])
df_sorted_one = df_sorted.drop_duplicates(subset='user_id', keep='first')[['user_id','created_at']]
df_sorted_one

,user_id,created_at
12411,838023,2023-04-29 16:22:56
55708,838466,2023-05-02 22:42:05
903,838642,2023-04-28 14:18:07
12717,839357,2023-04-29 16:51:01
57398,840293,2023-05-03 02:24:05
...,...,...
1008979,1510561,2023-05-25 14:28:20
1011373,1511957,2023-05-25 15:46:25
1012114,1512422,2023-05-25 16:57:53
1014009,1512542,2023-05-25 22:59:47


In [34]:
user_list = df.user_id.unique()
user_df = pd.DataFrame(user_list, columns=['user_id'])
user_df

,user_id
0,849436
1,847375
2,849438
3,849479
4,849441
...,...
4698,1476931
4699,1511957
4700,1512422
4701,1512542


In [ ]:
user_created = (
    ds.votes.accounts_user()
    .query("id in @user_list")
)

In [58]:
# 포인트 사용 테이블 읽어오기
point_history_df = ds.votes.accounts_pointhistory().loc[ds.votes.accounts_pointhistory()['delta_point'] < 0]
point_history_df

,id,delta_point,created_at,user_id,user_question_record_id
1030,808783,-300,2023-04-28 14:31:21,849479,773997.0
1036,808861,-300,2023-04-28 14:31:49,849452,788254.0
1110,810295,-300,2023-04-28 14:41:47,849762,787307.0
1241,812641,-300,2023-04-28 14:55:08,849670,782368.0
1443,815628,-300,2023-04-28 15:15:43,849439,790323.0
...,...,...,...,...,...
2338838,340661110,-500,2024-04-30 10:33:03,1209776,70382111.0
2338839,340661111,-1000,2024-04-30 10:33:13,1209776,70382111.0
2338870,340668595,-200,2024-05-05 14:06:53,1251933,89164250.0
2338906,340671283,-500,2024-05-07 02:26:58,851491,3774736.0


In [59]:
point_history_df = (
    point_history_df
    .sort_values(by=['user_id','created_at'])
    .drop_duplicates(subset='user_id')
)
point_history_df

,id,delta_point,created_at,user_id,user_question_record_id
27111,1360127,-300,2023-04-30 14:27:35,838023,1281323.0
170410,5055264,-300,2023-05-05 07:32:34,838466,3524303.0
19162,1182553,-300,2023-04-30 06:49:01,838642,1068384.0
16113,1107427,-300,2023-04-30 02:19:41,839357,1035633.0
210125,5596643,-300,2023-05-05 14:49:11,840293,3798163.0
...,...,...,...,...,...
2272250,318278139,-200,2023-06-08 08:31:10,1571821,151205021.0
2333926,339377675,-200,2023-09-20 13:12:05,1576039,161095445.0
2302626,328376445,-200,2023-06-17 11:04:28,1576107,155931559.0
2314798,332869984,-200,2023-06-30 06:11:37,1576705,158223837.0


In [ ]:
# 지표 dataframe 생성

# 회원가입한 날짜와 연결
working_merge = pd.merge(
    user_df,
    user_created[['id','created_at']],
    left_on='user_id',
    right_on='id'
).drop(columns='id')

working_merge = working_merge.rename(columns={'created_at':'sign_up_date'})

# 처음 투표한 날짜와 연결
working_merge = pd.merge(
    working_merge,
    df_sorted_one,
    on = 'user_id'
)
working_merge = working_merge.rename(columns={'created_at':'first_vote_date'})

# 처음 포인트 쓴 날과 양 연결
working_merge = pd.merge(
    working_merge,
    point_history_df[['user_id','created_at','delta_point']],
    on = 'user_id',
    how='left'
)
working_merge = working_merge.rename(columns={'created_at':'first_point_use_date', 'delta_point':'first_point_use_amount'})


# 중간 결과물
working_merge

,user_id,sign_up_date,first_vote_date,first_point_use_date,first_point_use_amount
0,849436,2023-04-28 02:54:59.981972,2023-04-28 12:27:49,2023-04-28 17:44:51,-300.0
1,847375,2023-04-26 11:32:36.470049,2023-04-28 12:28:46,2023-04-29 09:09:36,-300.0
2,849438,2023-04-28 02:56:02.571340,2023-04-28 12:29:29,2023-04-30 13:37:52,-300.0
3,849479,2023-04-28 03:27:32.406440,2023-04-28 12:29:52,2023-04-28 14:31:21,-300.0
4,849441,2023-04-28 02:56:56.323854,2023-04-28 12:30:06,NaT,NaN
...,...,...,...,...,...
4698,1476931,2023-05-23 13:54:41.653078,2023-05-25 14:39:42,2023-05-29 15:14:23,-10.0
4699,1511957,2023-05-25 15:45:50.814710,2023-05-25 15:46:25,NaT,NaN
4700,1512422,2023-05-25 16:52:43.081341,2023-05-25 16:57:53,2023-05-25 17:09:17,-10.0
4701,1512542,2023-05-25 20:03:38.172395,2023-05-25 22:59:47,2023-05-26 11:28:52,-200.0


In [63]:
# 날짜형으로 변환 (혹시 이미 datetime이면 생략 가능)
working_merge['sign_up_date'] = pd.to_datetime(working_merge['sign_up_date'])
working_merge['first_vote_date'] = pd.to_datetime(working_merge['first_vote_date'])
working_merge['first_point_use_date'] = pd.to_datetime(working_merge['first_point_use_date'])

working_merge['vote_within_signup_1days'] = (
    ((working_merge['first_vote_date'] - working_merge['sign_up_date']).dt.days <= 1)
    & working_merge['first_vote_date'].notnull()
).astype(int)


working_merge['point_used_within_signup_3days'] = (
    ((working_merge['first_point_use_date'] - working_merge['sign_up_date']).dt.days <= 3)
    & working_merge['first_point_use_date'].notnull()
).astype(int)

In [65]:
working_merge.to_csv('./use_point_within_signup_N_days.csv', encoding='UTF-8', index=False)